In [1]:
import pyrealsense2 as rs
import ipywidgets
import time
import struct
import serial
import queue
ser = serial.Serial("/dev/ttyAMA0", 230400) 

freq = 30

# Declare RealSense pipeline, encapsulating the actual device and sensors
pipe = rs.pipeline()

# Build config object and request pose data
cfg = rs.config()
cfg.enable_stream(rs.stream.pose)

# Start streaming with requested config
pipe.start(cfg)

lasttime=time.time()
num=0
rolist=[[0,0,0],[1,0,0],[0,1,0],[0,1,1],[0,0,1],[0,0,0],[2,0,0]]
roqu=queue.Queue()
#起飞标志 X坐标 Y坐标 高度 TIME
roqu.put([0,0,0,1.5,5])#起飞前等待
roqu.put([1,0,0,1.5,5])#起飞
roqu.put([0,0.5,2,1.5,5])#21
roqu.put([0,0.5,2.5,1.5,5])#28
roqu.put([0,1,2.5,1.5,5])#27
roqu.put([0,1,2,1.5,5])#20
roqu.put([0,1.5,2,1.5,5])#19
roqu.put([0,1.5,2.5,1.5,5])#26
roqu.put([0,2,2.5,1.5,5])#25
roqu.put([0,2.5,2.5,1.5,5])#24
roqu.put([0,3,2.5,1.5,5])#23
roqu.put([0,3.5,2.5,1.5,5])#22
roqu.put([0,3.5,2,1.5,5])#15
roqu.put([0,3,2,1.5,5])#16
roqu.put([0,2.5,2,1.5,5])#17
roqu.put([0,2,2,1.5,5])#18
roqu.put([0,2,1.5,1.5,5])#14
roqu.put([0,2.5,1.5,1.5,5])#13
roqu.put([0,3,1.5,1.5,5])#12
roqu.put([0,3.5,1.5,1.5,7])#11
roqu.put([0,2.5,1,1.5,5])#9
roqu.put([0,2,1,1.5,5])#10
roqu.put([0,2,0.5,1.5,5])#8
roqu.put([0,2.5,0.5,1.5,5])#7
roqu.put([0,3,0.5,1.5,5])#6
roqu.put([0,3.5,0.5,1.5,5])#5
roqu.put([0,3.5,0,1.5,5])#1
roqu.put([0,3,0,1.5,5])#2
roqu.put([0,2.5,0,1.5,5])#3
roqu.put([0,2,0,1.5,5])#4
roqu.put([0,0,0,1.5,10])#home
roqu.put([2,0,0,1.5,5])#降落
wattime=0
try:
    while 1:
        # Wait for the next set of frames from the camera
        frames = pipe.wait_for_frames()

        # Fetch pose frame
        pose = frames.get_pose_frame()
        if pose:
            # Print some of the pose data to the terminal
            data = pose.get_pose_data()
            #print("Frame #{}".format(pose.frame_number))
            #print("Position: {}".format(data.translation))
            #print("Velocity: {}".format(data.velocity))
            #print("Acceleration: {}\n".format(data.acceleration))
            '''if(time.time()>lasttime+5):
                num+=1
                lasttime=time.time()'''
            
            #dron=rolist[num][0]
            #tarx=rolist[num][1]
            #tary=rolist[num][2]
            if(time.time()>lasttime+wattime):
                if not roqu.empty():
                    rott=roqu.get()
                else:
                    #pipe.stop()
                    break
                lasttime=time.time()
            dron=rott[0]
            tarx=rott[1]
            tary=rott[2]
            height=rott[3]
            wattime=rott[4]
            ub=struct.pack('BBBBfff',0x55,0xAA,dron,0,tarx-data.translation.z,tary-data.translation.x,height)
            ser.write(ub) 
            
            if(dron==2):
                #pipe.stop()
                break
            print("\rPosition: {} {}".format(data.translation,rott),end="")
            
        #time.sleep(1/freq)

finally:
    pipe.stop()
    ub=struct.pack('BBBBfff',0x55,0xAA,2,0,0,0,1)
    ser.write(ub) 

Position: x: 0.0230565, y: 0.997708, z: -0.00133242 [0, 0, 0, 1.5, 10]]]]]]]]

RuntimeError: stop() cannot be called before start()